# Pandas Dataframe

This notebook shows how to use agents to interact with a `Pandas DataFrame`. It is mostly optimized for question answering.

**NOTE: this agent calls the `Python` agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**

In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [2]:
import pandas as pd
from langchain_openai import OpenAI

df = pd.read_excel(
    "../files/display-long-term-demand-forecast-tracker-pivot-q1-2016(1)_1.xls",
    sheet_name="Units",
    skiprows=[i for i in range(13)],
    header=None
)

In [4]:
import numpy as np

In [5]:
def format_year(row, start_year):
    j = 0
    years = []
    for i in range(start_year, len(row)):
        if not np.isnan(row[i]):
            j = i
        years.append(str(row[j]))
    formated_row = years
    return formated_row

def format_year_quater(years, quaters):
    assert len(years) == len(quaters)
    return ["-".join([years[i], quaters[i]]) for i in range(len(years))]
def fill_attribute(column):
    column = list(column)
    j = 0
    new_column = []
    for i in range(len(column)):
        if isinstance(column[i], float) and np.isnan(column[i]):
            pass
        else:
            j = i
        new_column.append(column[j])
    return new_column

In [6]:
def format_df(df, value_unit, num_col2fill):
    years = format_year(df.iloc[0], num_col2fill)
    years_quaters = format_year_quater(years, df.iloc[1, num_col2fill:].to_list())
    col2fill = df.iloc[1, :num_col2fill].to_list()
    df.columns = col2fill + [" ".join([yq, value_unit]) for yq in years_quaters]
    df = df.iloc[2:].reset_index(drop=True)
    for c in range(num_col2fill):
        df.iloc[:, c] = fill_attribute(df.iloc[:, c])
    return df

def format_df_t(df):
    years = df.iloc[[0]].apply(format_year, axis=1).iloc[0]
    years_quaters = format_year_quater(years, df.iloc[1, 1:].to_list())
    df.columns = ['year quatuer'] + years_quaters
    _df = df.iloc[2:].reset_index(drop=True).T
    _df.reset_index(inplace=True)
    _df.columns = _df.iloc[0]
    _df = _df.drop(0)
    _df = _df.reset_index(drop=True)
    return _df

In [7]:
df_1 = format_df(df, 'Units', 3)
df_1

,Original_Specification,Resolution,Size,2013-Q1 Units,2013-Q2 Units,2013-Q3 Units,2013-Q4 Units,2014-Q1 Units,2014-Q2 Units,2014-Q3 Units,...,2019-Q1 Units,2019-Q2 Units,2019-Q3 Units,2019-Q4 Units,2020-Q1 Units,2020-Q2 Units,2020-Q3 Units,2020-Q4 Units,2021-ALL Units,2022-ALL Units
0,3D Shutter,Segment,2.5,6230,8288,4182,4264,3296,1160,881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amusement,320 x 240,3.1,660,700,550,900,30,90,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amusement,320 x 240,3.3,NaN,NaN,NaN,NaN,NaN,50,900,...,77.774471,90.05465,128.941886,84.596793,75.783774,87.749633,125.64152,82.431473,356.796,343.332
3,Amusement,320 x 240,4.2,1700,1700,4000,2200,460,220,1350,...,820,1150,2220,1970,799.366883,1121.063312,2164.13961,1920.430195,5880,5760.0
4,Amusement,400 x 480,5,NaN,NaN,2100,1900,500,60,400,...,50,100,150,60,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,Others,1280 x 1024,17,71.745,75,118,463,663,484,375,...,319.313,330.007,377.809,376.799,348.104,376.168,403.739,410.591,1812.818,2161.641
656,Others,1920 x 1080,16.5,4,0.5,0.3,NaN,1,3,1,...,2,2,2,2,2.125,2.125,2.125,2.125,9,9
657,Others,1920 x 1080,24.5,3,0.4,0.2,NaN,NaN,5,3.6,...,1.5,1.5,2.5,2.5,1.59375,1.59375,2.65625,2.65625,9,9.5
658,Others,4096 x 2160,29.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.5,1,1,1,0.5,1,1,1,4.0,4.0


## Using `ZERO_SHOT_REACT_DESCRIPTION`

This shows how to initialize the agent using the `ZERO_SHOT_REACT_DESCRIPTION` agent type. Note that this is an alternative to the above.

In [8]:
import os
os.environ['OPENAI_API_KEY'] = "token"

In [9]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

## Using OpenAI Functions

This shows how to initialize the agent using the OPENAI_FUNCTIONS agent type. Note that this is an alternative to the above.

In [10]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df_1,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [11]:
agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


27There are 27 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 27 rows in the dataframe.'}

In [18]:
agent.invoke("What is the total value of ctr tv and lcd tv in 2015 all quaters")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.loc[4, '2015-Q1':'2015-Q4'].sum()"}`


1135.99473575The total value of CRT TV and LCD TV in all quarters of 2015 is 1135.99473575.

> Finished chain.


{'input': 'What is the total value of ctr tv and lcd tv in 2015 all quaters',
 'output': 'The total value of CRT TV and LCD TV in all quarters of 2015 is 1135.99473575.'}

In [14]:
agent.invoke("whats the average value of Mobile Phone?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Mobile Phone'].mean()"}`


593442.9088441757The average value of Mobile Phone is approximately 593,442.91.

> Finished chain.


{'input': 'whats the average value of Mobile Phone?',
 'output': 'The average value of Mobile Phone is approximately 593,442.91.'}

In [19]:
_df[_df['year quatuer'].str.contains('2016')]['LCD TV'].values[0]

59606.907

In [27]:
agent.invoke("How much is the growth of LCD TV from 2013 Q1 to 2016 Q4? And the compound growth rate")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['LCD TV'][15] - df['LCD TV'][0]"}`


11444.132999999987
Invoking: `python_repl_ast` with `{'query': "(df['LCD TV'][15] - df['LCD TV'][0]) / df['LCD TV'][0] * 100"}`


20.071792128525303
Invoking: `python_repl_ast` with `{'query': "((df['LCD TV'][15] / df['LCD TV'][0])**(1/15) - 1) * 100"}`


1.2269300868725797The growth of LCD TV from 2013 Q1 to 2016 Q4 is 11,444 units, which represents a growth rate of 20.07%. The compound growth rate over this period is 1.23%.

> Finished chain.


{'input': 'How much is the growth of LCD TV from 2013 Q1 to 2016 Q4? And the compound growth rate',
 'output': 'The growth of LCD TV from 2013 Q1 to 2016 Q4 is 11,444 units, which represents a growth rate of 20.07%. The compound growth rate over this period is 1.23%.'}

In [26]:
agent.invoke("What is the compound growth rate of LCD TV from 2013 Q1 to 2016 Q4?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "beginning_value = df.loc[0, 'LCD TV']\nending_value = df.loc[11, 'LCD TV']\nnumber_of_periods = 11\ncompound_growth_rate = (ending_value / beginning_value) ** (1 / number_of_periods) - 1\ncompound_growth_rate"}`
responded: To calculate the compound growth rate of LCD TV from 2013 Q1 to 2016 Q4, we need to use the formula:

Compound Growth Rate = (Ending Value / Beginning Value) ^ (1 / Number of Periods) - 1

In this case, the beginning value is the LCD TV sales in 2013 Q1 and the ending value is the LCD TV sales in 2016 Q4. The number of periods is the number of quarters between 2013 Q1 and 2016 Q4.

Let's calculate the compound growth rate using the data from the dataframe.

0.017875395725760912The compound growth rate of LCD TV from 2013 Q1 to 2016 Q4 is approximately 1.79%.

> Finished chain.


{'input': 'What is the compound growth rate of LCD TV from 2013 Q1 to 2016 Q4?',
 'output': 'The compound growth rate of LCD TV from 2013 Q1 to 2016 Q4 is approximately 1.79%.'}

In [23]:
agent.invoke("Which size of desktop monitor experienced the fastest growth from 2013 Q1 to 2017 Q4")



> Entering new AgentExecutor chain...
To determine which size of desktop monitor experienced the fastest growth from 2013 Q1 to 2017 Q4, we need to compare the sales data for each size of desktop monitor across the different quarters.

First, we need to extract the relevant columns from the dataframe and filter the data for the specified time period. Then, we can calculate the growth rate for each size of desktop monitor and identify the one with the highest growth rate.

Here's the Python code to accomplish this:

```python
import pandas as pd

# Extract relevant columns and filter data for the specified time period
df_filtered = df[['year quatuer', 'Desktop Monitor']].copy()
df_filtered = df_filtered[df_filtered['year quatuer'].between('2013-Q1', '2017-Q4')]

# Calculate growth rate for each size of desktop monitor
df_filtered['Growth Rate'] = df_filtered['Desktop Monitor'].pct_change()

# Find the size of desktop monitor with the highest growth rate
fastest_growth_size = df_filter

{'input': 'Which size of desktop monitor experienced the fastest growth from 2013 Q1 to 2017 Q4',
 'output': "To determine which size of desktop monitor experienced the fastest growth from 2013 Q1 to 2017 Q4, we need to compare the sales data for each size of desktop monitor across the different quarters.\n\nFirst, we need to extract the relevant columns from the dataframe and filter the data for the specified time period. Then, we can calculate the growth rate for each size of desktop monitor and identify the one with the highest growth rate.\n\nHere's the Python code to accomplish this:\n\n```python\nimport pandas as pd\n\n# Extract relevant columns and filter data for the specified time period\ndf_filtered = df[['year quatuer', 'Desktop Monitor']].copy()\ndf_filtered = df_filtered[df_filtered['year quatuer'].between('2013-Q1', '2017-Q4')]\n\n# Calculate growth rate for each size of desktop monitor\ndf_filtered['Growth Rate'] = df_filtered['Desktop Monitor'].pct_change()\n\n# Find th

In [17]:
agent.invoke("The LCD TV with 4k resolution from 2016 to 2019, respectively, was how much?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.loc[(df['Resolution'] == '4k') & (df['Size'] == 4), '2016-Q1 Units':'2019-Q4 Units'].sum().sum()"}`


0The total number of LCD TVs with 4k resolution and a size of 4 from 2016 to 2019 is 0.

> Finished chain.


{'input': 'The LCD TV with 4k resolution from 2016 to 2019, respectively, was how much?',
 'output': 'The total number of LCD TVs with 4k resolution and a size of 4 from 2016 to 2019 is 0.'}

不知道 4k 代表什么

In [15]:
agent.invoke("The value of LCD TV with 3840 x 2160 resolution from 2016 to 2019, respectively, was how much?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.loc[(df['Original_Specification'] == 'LCD TV') & (df['Resolution'] == '3840 x 2160'), '2016-Q1 Units':'2019-Q4 Units'].sum().sum()"}`


376371.641The value of LCD TVs with a resolution of 3840 x 2160 from 2016 to 2019 was approximately 376,371.641 units.

> Finished chain.


{'input': 'The value of LCD TV with 3840 x 2160 resolution from 2016 to 2019, respectively, was how much?',
 'output': 'The value of LCD TVs with a resolution of 3840 x 2160 from 2016 to 2019 was approximately 376,371.641 units.'}

没有分别统计而是 sum 起来了

In [18]:
agent.invoke("Which size of desktop monitors experienced the fastest growth from 2013 to 2017?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "growth_rates = {}\n\nfor index, row in df.iterrows():\n    if pd.notnull(row['Size']):\n        units_2013 = row['2013-Q1 Units'] + row['2013-Q2 Units'] + row['2013-Q3 Units'] + row['2013-Q4 Units']\n        units_2017 = row['2017-Q1 Units'] + row['2017-Q2 Units'] + row['2017-Q3 Units'] + row['2017-Q4 Units']\n        growth_rate = ((units_2017 - units_2013) / units_2013) * 100\n        growth_rates[row['Size']] = growth_rate\n\nmax_growth_size = max(growth_rates, key=growth_rates.get)\nmax_growth_size"}`
responded: To determine which size of desktop monitors experienced the fastest growth from 2013 to 2017, we need to calculate the growth rate for each size and compare them.

First, we need to calculate the growth rate for each size by subtracting the units sold in 2013 from the units sold in 2017, and then dividing it by the units sold in 2013. We will use the following formula:

Growth Rate = ((Uni

{'input': 'Which size of desktop monitors experienced the fastest growth from 2013 to 2017?',
 'output': 'The size of desktop monitors that experienced the fastest growth from 2013 to 2017 is 82.'}

## Multi DataFrame Example

This next part shows how the agent can interact with multiple dataframes passed in as a list.

In [ ]:
_df = pd.read_excel(
    "../files/display-long-term-demand-forecast-tracker-pivot-q1-2016(1)_1.xls",
    sheet_name="Revenues",
    skiprows=[i for i in range(13)],
    header=None
)

df_2 = format_df(_df, 'Revenues', 1)

In [ ]:
df1, df2 = df_1, df_2
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"), 
    [df1, df2], 
    verbose=True,
    max_iterations=10,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [32]:
agent.invoke("how much is the revenues of lcd tv in 2015")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1[df1['Original_Specification'] == 'LCD TV']['2015-Q1 Revenues'].values[0]"}`


KeyError: '2015-Q1 Revenues'
Invoking: `python_repl_ast` with `{'query': "df1[df1['Original_Specification'] == 'LCD TV']['2015-Q2 Revenues'].values[0] + df1[df1['Original_Specification'] == 'LCD TV']['2015-Q3 Revenues'].values[0] + df1[df1['Original_Specification'] == 'LCD TV']['2015-Q4 Revenues'].values[0]"}`


KeyError: '2015-Q2 Revenues'
Invoking: `python_repl_ast` with `{'query': "df1[df1['Original_Specification'] == 'LCD TV'][['2015-Q1 Revenues', '2015-Q2 Revenues', '2015-Q3 Revenues', '2015-Q4 Revenues']].sum(axis=1).values[0]"}`


KeyError: "None of [Index(['2015-Q1 Revenues', '2015-Q2 Revenues', '2015-Q3 Revenues',\n       '2015-Q4 Revenues'],\n      dtype='object')] are in the [columns]"
Invoking: `python_repl_ast` with `{'query': "df2[df2['Original_Specification'] == 'LCD TV'][['2015-Q1 Revenues', '2015-Q2 Reve

{'input': 'how much is the revenues of lcd tv in 2015',
 'output': 'The revenues of LCD TV in 2015 is approximately $40,942,315.91.'}

In [28]:
agent.invoke("Which size of desktop monitors experienced the fastest growth in Units from 2013 to 2017? And what was the increase in revenue from what to what?")



> Entering new AgentExecutor chain...
To determine which size of desktop monitors experienced the fastest growth in units from 2013 to 2017, we need to calculate the percentage increase in units for each size and compare them.

First, let's calculate the percentage increase in units for each size from 2013 to 2017:

```python
import pandas as pd

# Assuming df1 is the dataframe containing the units data
df1['2013-2017 Growth'] = (df1['2017-Q4 Units'] - df1['2013-Q1 Units']) / df1['2013-Q1 Units'] * 100

# Sort the dataframe by the growth rate in descending order
df1_sorted = df1.sort_values('2013-2017 Growth', ascending=False)

# Get the size with the highest growth rate
fastest_growth_size = df1_sorted.iloc[0]['Original_Specification']
fastest_growth_rate = df1_sorted.iloc[0]['2013-2017 Growth']
```

The `fastest_growth_size` variable will contain the size of the desktop monitor that experienced the fastest growth in units from 2013 to 2017, and the `fastest_growth_rate` variable wi

{'input': 'Which size of desktop monitors experienced the fastest growth in Units from 2013 to 2017? And what was the increase in revenue from what to what?',
 'output': "To determine which size of desktop monitors experienced the fastest growth in units from 2013 to 2017, we need to calculate the percentage increase in units for each size and compare them.\n\nFirst, let's calculate the percentage increase in units for each size from 2013 to 2017:\n\n```python\nimport pandas as pd\n\n# Assuming df1 is the dataframe containing the units data\ndf1['2013-2017 Growth'] = (df1['2017-Q4 Units'] - df1['2013-Q1 Units']) / df1['2013-Q1 Units'] * 100\n\n# Sort the dataframe by the growth rate in descending order\ndf1_sorted = df1.sort_values('2013-2017 Growth', ascending=False)\n\n# Get the size with the highest growth rate\nfastest_growth_size = df1_sorted.iloc[0]['Original_Specification']\nfastest_growth_rate = df1_sorted.iloc[0]['2013-2017 Growth']\n```\n\nThe `fastest_growth_size` variable w

没有调用 python_repl_ast